# Meteoros brilhantes - Exemplo prático 1

As bolas de fogo e os sólidos são termos astronómicos para meteoros excepcionalmente brilhantes que são espetaculares o suficiente para serem vistos numa área muito vasta. [Um conjunto de dados da NASA](https://www.kaggle.com/nasa/fireballs) registra os eventos mais vísiveis, e permite que a gente levante algumas questões interessante:
* Existe algum padrão na velocidade dos meteoritos?
* A altitude percebida na hora de maior brilho influencia na energia?

Vamos importar nossas bibliotecas para responder essas perguntas.

In [1]:
import pandas as pd
import altair as alt

In [2]:
url = "https://raw.githubusercontent.com/tiagodavi70/nasaFireballs/master/cneos_fireball_data.csv"

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/tiagodavi70/nasaFireballs/master/cneos_fireball_data.csv")

## Existe algum padrão na velocidade dos meteoritos?

A velocidade registrada dos meteoritos pode ser apresentada de forma agrupada, com um histograma simples.

In [4]:
histograma_velocidade = alt.Chart(url).mark_bar().encode(
    alt.X("Velocity (km/s):Q", bin=True, title="Velocidade em km/s"),
    alt.Y("count()", title="Contagem"),
    color=alt.value("SandyBrown")
)
histograma_velocidade

alt.Chart(...)

Existe uma distribuição quase normal, com alguma asimetria. Podemos adicionar as datas em que foram registradas, para tentar relacionar a velocidade durante os anos.

In [5]:
alt.Chart(url).mark_line(opacity=.5, point=True).encode(
    alt.X("Peak Brightness Date/Time (UT):T", title=""),
    alt.Y("Velocity (km/s):Q", title="Velocidade em km/s"),
    color=alt.value("SandyBrown"),
    tooltip=["mean(Velocity (km/s)):Q"]
).transform_filter(
    'datum["Velocity (km/s)"] !== \"\"' # removendo valores vazios
)

alt.Chart(...)

Conseguimos ver um conjunto de pontos agrupados de maneira geral, mas será que existe sazonalidade?

In [6]:
selecao_mes = alt.selection_interval(empty="all")

ano_mes = alt.Chart(url).mark_rect(opacity=1).encode(
    alt.Y("month(Peak Brightness Date/Time (UT)):T", title=""),
    alt.X("year(Peak Brightness Date/Time (UT)):T", title=""),
    color=alt.condition(selecao_mes, "mean(Velocity (km/s)):Q", alt.value("grey")),
    tooltip=["mean(Velocity (km/s)):Q"]
).transform_filter(
    'datum["Velocity (km/s)"] !== \"\"' # removendo valores vazios
).add_selection(selecao_mes)

dia_hora = alt.Chart(url).mark_rect(opacity=1).transform_filter(
    selecao_mes
).encode(
    alt.Y("date(Peak Brightness Date/Time (UT)):T", title="Dia do mês"),
    alt.X("hours(Peak Brightness Date/Time (UT)):T", title=""),
    alt.Color("mean(Velocity (km/s)):Q", title="Média da Velocidade em km/s", scale=alt.Scale(scheme="reds")),
    tooltip=["Velocity (km/s):Q"]
).transform_filter(
    'datum["Velocity (km/s)"] !== \"\"' # removendo valores vazios
)

(ano_mes | dia_hora).properties(title="Arraste o mouse no gráfico abaixo para refletir a direita")

alt.HConcatChart(...)

A velocidade dos meteoros não tem nenhum padrão claro de sazonalidade, nem em relação aos anos e nem do horário.

## A altitude percebida na hora de maior brilho influencia na energia?

A altitude registrada no conjunto de dados é calculada com base no momento de maior brilho do meteorito. Vamos investigar se existe uma relação entre essa altitude registrada e as energias presentes no conjunto de dados. Antes disso, é importante ver como as energias se relacionam. Para isso, vamos procurar uma relação.

In [7]:
alt.Chart(url).mark_point(opacity=.6).encode(
    alt.X('Total Radiated Energy (J):Q'),
    alt.Y("Calculated Total Impact Energy (kt):Q"),
    color=alt.value("SandyBrown")
).transform_filter(
    "datum['Calculated Total Impact Energy (kt)'] < 400" # removendo um outlier
).interactive()

alt.Chart(...)

Um ponto registrado tem valores absurdamente altos para a distribuição, e foi retirado aqui e será retirado em todos os gráficos daqui pra frente. Ainda tem alguns outros pontos com valores altos, mas para essa relação podemos ver claramente que as energias são proporcinais. Então vamos usar somente uma delas para uma visualização interativa.

In [8]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
)

histograma = alt.Chart().mark_rect(color="SandyBrown", width=3).encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100)),
    alt.Y("count()", title="Contagem")
).transform_filter(
    'datum["Altitude (km)"] !== \"\"' # removendo valores vazios
)

hists = alt.layer(
    histograma.add_selection(brush).encode(color=alt.value('lightgrey')),
    histograma.transform_filter(brush)
).properties(
    width=400,
    height=100
).repeat(
    row=["Calculated Total Impact Energy (kt)", "Altitude (km)"],
    data=url
)

scatterplot = alt.Chart(url).mark_point(stroke="darkorange", opacity=.5).encode(
    alt.X("Calculated Total Impact Energy (kt):Q",scale=alt.Scale(type="log", domain=[0.03, 10]), title="Energia (escala logarítmica)"),
    alt.Y("Altitude (km):Q",scale=alt.Scale(domain=[0, 80])),
    tooltip=["Calculated Total Impact Energy (kt):N", "Altitude (km):N"]
).transform_filter(brush)

(hists | scatterplot).configure_view(
    stroke='transparent' # sem outline
).transform_filter(
    "datum['Calculated Total Impact Energy (kt)'] < 10 && datum['Altitude (km)'] !== \"\"" # removendo outliers
).properties(
    title="Arraste o mouse nos histogramas para refletir no scatterplot"
)

alt.HConcatChart(...)

Retirando grandes valores, podemos ver melhor a distribuição, assim como mudar a escala. Podemos observar que não existe uma relação direta entre a altitude registrada e a energia radiada. 